# Les imports

In [1]:
from __future__ import division
import os
import urllib2

In [2]:
import numpy as np
import pandas as pd
pd.core.series.Series.value_counts.im_func.func_defaults = (False, False, False, None, False)
pd.set_option('display.max_rows', 200)
pd.set_option('display.max_columns', 100)
#pd.set_option('display.width', 200)
#pd.set_option('display.height', 100)

## Fichiers dbf à HDF

In [4]:
from simpledbf import Dbf5
%time #TODO : automatic download for a given link  http://www.insee.fr/fr/themes/detail.asp?ref_id=fd-eec12&page=fichiers_detail/eec12/telechargement.htm
table1_dbf = Dbf5('/Users/adrienpacifico/Informatique/Economie/enquete_emploi_stat_des/eec11_indiv11_dbase/indiv111.dbf')
table2_dbf = Dbf5('/Users/adrienpacifico/Informatique/Economie/enquete_emploi_stat_des/eec11_indiv11_dbase/indiv112.dbf')
table3_dbf = Dbf5('/Users/adrienpacifico/Informatique/Economie/enquete_emploi_stat_des/eec11_indiv11_dbase/indiv113.dbf'); table3_dbf

CPU times: user 2 µs, sys: 0 ns, total: 2 µs
Wall time: 5.01 µs


In [5]:
assert table3_dbf.numrec == table2_dbf.numrec == table1_dbf.numrec 

In [8]:
table1_dbf.numrec 

423425

In [9]:
file_path = "/Users/adrienpacifico/Informatique/Economie/enquete_emploi_stat_des/"

In [10]:
%%time
if not os.path.isfile(file_path + 'indiv111.csv'): #Do not create csv file if allready exists
    table1_dbf.to_csv(file_path +'indiv111.csv')
if not os.path.isfile(file_path +'indiv112.csv'):    
    table2_dbf.to_csv(file_path +'indiv112.csv')
if not os.path.isfile(file_path +'indiv113.csv'): 
    table3_dbf.to_csv(file_path +'indiv113.csv')

CPU times: user 28 µs, sys: 67 µs, total: 95 µs
Wall time: 97 µs


In [13]:
indivi111 = file_path +'indiv111.csv'
! wc -l $indivi111

  423426 /Users/adrienpacifico/Informatique/Economie/enquete_emploi_stat_des/indiv111.csv


In [15]:
%%time
if not os.path.isfile(file_path +'eec_2011.h5'):
    print 'csv are read'
    table1 = pd.read_csv(file_path +'indiv111.csv', low_memory=False)
    table2 = pd.read_csv(file_path +'indiv112.csv', low_memory=False)
    table3 = pd.read_csv(file_path +'indiv113.csv', low_memory=False)
    
    assert (
            table1.shape[0] == table2.shape[0] == table3.shape[0]#all tables must have the same number of raws
            == table3_dbf.numrec == table2_dbf.numrec == table1_dbf.numrec # and the save nb of raws as the original dbf file  
           ) 
    print table1.shape[0], table2.shape[0], table3.shape[0]

CPU times: user 93 µs, sys: 81 µs, total: 174 µs
Wall time: 110 µs


In [13]:
%%time
if not os.path.isfile(file_path +'eec_2011.h5'):
    table1.to_hdf(file_path +'eec_2011.h5', 'table1', append = True) 
    table2.to_hdf(file_path +'eec_2011.h5', 'table2', append = True)
    table3.to_hdf(file_path +'eec_2011.h5', 'table3', append = True)

CPU times: user 58 µs, sys: 45 µs, total: 103 µs
Wall time: 74.9 µs


In [16]:
%%time
table_1 = pd.read_hdf(file_path +'eec_2012.h5', 'table1')
table_2 = pd.read_hdf(file_path +'eec_2012.h5', 'table2')
table_3 = pd.read_hdf(file_path +'eec_2012.h5', 'table3')

CPU times: user 13.6 s, sys: 5.82 s, total: 19.4 s
Wall time: 25.5 s


### Controle que les tables fassent la même taille

In [17]:
#if 'table1' in locals():
#assert table1.shape[0] == table2.shape[0] == table3.shape[0] == table_1.shape[0] == table_2.shape[0] == table_3.shape[0]
print table_1.shape[0]
print table_2.shape[0]
print table_3.shape[0]

422133
422133
422133


In [18]:
table1 = table_1
table2 = table_2
table3 = table_3

In [19]:
table1.ANNEE.value_counts()

2012    422133
dtype: int64

## Appariement des 3 sous-tables

In [20]:
#pd.merge(table1,table2, on = ('TRIM', 'IDENT','NOI'))
table1['TRIM'] = table1.TRIM.astype('str')
table1['IDENT'] = table1.IDENT.astype('str')
table1['NOI'] = table1.NOI.astype('str')

table2['TRIM'] = table2.TRIM.astype('str')
table2['IDENT'] = table2.IDENT.astype('str')
table2['NOI'] = table2.NOI.astype('str')

table3['TRIM'] = table3.TRIM.astype('str')
table3['IDENT'] = table3.IDENT.astype('str')
table3['NOI'] = table3.NOI.astype('str')


In [21]:
%%time
table1["id_unique"] = table1.IDENT + table1.TRIM + table1.NOI   #unique id for a given rank of interview
table2["id_unique"] = table2.IDENT + table2.TRIM + table2.NOI
table3["id_unique"] = table3.IDENT + table3.TRIM + table3.NOI
table1["id_indiv"] = table1.IDENT + table1.NOI  #unique id for a given individual
table2["id_indiv"] = table2.IDENT + table2.NOI
table3["id_indiv"] = table3.IDENT + table3.NOI
result = pd.merge(table1, table2, on='id_unique')
result = pd.merge(result, table3, on='id_unique')


CPU times: user 5.81 s, sys: 5.28 s, total: 11.1 s
Wall time: 13.3 s


In [22]:
print result.EXTRI113.sum()/10**6
print result.EXTRI1613.sum()/10**6
print result.EXTRI13.sum()/10**6

50.4131455875
50.4127540303
50.4122227411


11 - Indépendants
12 - Employeurs
13 - Aides familiaux
21 - Intérimaires
22 - Apprentis
33 - CDD (hors Etat, coll.loc.), hors contrats aides
34 - Stagiaires et contrats aides (hors Etat, coll.loc.)
35 - Autres contrats (hors Etat, coll.loc.)
43 - CDD (Etat, coll.loc.), hors contrats aides
44 - Stagiaires et contrats aides (Etat, coll.loc.)
45 - Autres contrats (Etat, coll.loc.) 

In [23]:
result.TRIM = result.TRIM.astype("int32")

In [24]:
#result.reset_index(inplace = True)
#for trim in range(1,5):
#    result.loc[result.TRIM == trim,'STC_{}'.format(trim) ] = result["STC"]
#    result.loc[result.TRIM == trim,'SP_{}'.format(trim) ] = result["SP00"]
#    result.loc[result.TRIM == trim,'STATUTP_{}'.format(trim) ] = result["STATUTP"]
#    result.loc[result.TRIM == trim,'STATUT_{}'.format(trim) ] = result["STATUT"]
#    
#    #print (result.TRIM == trim).value_counts()
#    print trim

In [25]:
result.reset_index(inplace = True)
var_list = list()

for var in ["STC", "SP00", "STATUTP", "STATUT"]:
    for trim in range(1,5):
        result.loc[result.TRIM == trim, var +'_{}'.format(trim) ] = result[var]
        var_list.append( var +'_{}'.format(trim))
        

In [26]:
result.set_index('id_indiv', inplace=True)

In [27]:
result.reset_index(inplace= True)
#df = result[["STC_1", "STC_2", "STC_3", "STC_4", "id_indiv", "EXTRI13", "EXTRI113", "EXTRI1613"]+["SP_{}".format(trim) for trim in range(1,5)]
#           ].groupby("id_indiv").sum()
df = result[var_list + [ "EXTRI13", "EXTRI113", "EXTRI1613", "id_indiv"]].groupby("id_indiv").sum()

In [28]:
df.head()

,STC_1,STC_2,STC_3,STC_4,SP00_1,SP00_2,SP00_3,SP00_4,STATUTP_1,STATUTP_2,STATUTP_3,STATUTP_4,STATUT_1,STATUT_2,STATUT_3,STATUT_4,EXTRI13,EXTRI113,EXTRI1613
id_indiv,,,,,,,,,,,,,,,,,,,
G002T0TB1,NaN,NaN,NaN,NaN,NaN,4,4,4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,225.685550,496.415270,248.207635
G002T0TB2,NaN,2,2,2,NaN,1,1,1,NaN,45,NaN,NaN,NaN,45,45,45,225.685550,496.415270,248.207635
G002T6DB1,NaN,NaN,NaN,2,NaN,NaN,NaN,1,NaN,NaN,NaN,45,NaN,NaN,NaN,45,70.940001,0.000000,0.000000
G002T6DB2,NaN,NaN,NaN,2,NaN,NaN,NaN,1,NaN,NaN,NaN,45,NaN,NaN,NaN,45,70.940001,0.000000,0.000000
G002TBXB1,NaN,NaN,NaN,NaN,NaN,4,4,4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,233.114929,471.626902,235.813451


In [29]:
df[(~df.STC_1.isnull()) & (~df.STC_4.isnull())].STC_4.value_counts(normalize = True)

1    0.120642
2    0.874053
3    0.005305
dtype: float64

In [30]:
df1  = df[(~df.SP00_1.isnull()) & (~df.SP00_4.isnull())]
df2  = df[(~df.STC_1.isnull()) & (~df.STC_4.isnull())]
df3  = df[(~df.STATUT_1.isnull()) & (~df.STATUT_4.isnull())]
print df1.shape , df2.shape , df3.shape, df.shape # df1 contient les individus renseignés sur les 4 trimestres, df contient les individus qui ne sont pas forcément renseigné sur les 4 trimestres 

(37323, 19) (21112, 19) (21118, 19) (171876, 19)


In [31]:
((df1.EXTRI13.sum())*8/3)/10**6 #Car on prends les 3/8 ème de la base

46.421183586449153

In [32]:
((df1.EXTRI13.sum())*8/3)/result.EXTRI13.sum() # On perd 8% du sample, je corrige en assumant qu'il n'y a pas de biais de sélection

0.92083191461104086

In [33]:
df1.EXTRI13 = df1.EXTRI13 * (1/(((df1.EXTRI13.sum())*8/3)/result.EXTRI13.sum()))*8/3

/usr/local/lib/python2.7/site-packages/pandas/core/generic.py:2177: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self[name] = value


In [34]:
df.EXTRI13.sum()

50412222.741060682

In [35]:
df = df1

### Changement de STC

In [36]:

for trim in range(2,5):
    changement_situation_mois = "changement_STC_{}".format(trim)
    sitmois_minus_one = "STC_{}".format(trim-1)
    sitmois = "STC_{}".format(trim)
    df[changement_situation_mois] = ((df[sitmois] != df[sitmois_minus_one]))



In [37]:
df[["changement_STC_{}".format(trim) for trim in range(2,5)]].sum(1).value_counts(normalize = True)

0    0.503684
1    0.024007
2    0.078343
3    0.393966
dtype: float64

### Changement de STATUT

In [38]:
for trim in range(2,5):
    changement_situation_mois = "changement_STATUT_{}".format(trim)
    sitmois_minus_one = "STATUT_{}".format(trim-1)
    sitmois = "STATUT_{}".format(trim)
    df[changement_situation_mois] = ((df[sitmois] != df[sitmois_minus_one]))

In [39]:
#is_independant = (df[["STC_{}".format(trim) for trim in range(1,5)]] ==1).sum(1)
is_independant = (df[["STATUT_{}".format(trim) for trim in range(1,5)]] ==11).sum(1)

In [40]:
df_indep = df[is_independant >0]

In [41]:
df_indep.shape

(1659, 25)

In [42]:
df_indep["changement_statut"] = df_indep[["changement_STATUT_{}".format(trim) for trim in range(2,5)]].sum(1).astype("bool")

/usr/local/lib/python2.7/site-packages/ipykernel/__main__.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':


In [43]:
df_indep.head()

,STC_1,STC_2,STC_3,STC_4,SP00_1,SP00_2,SP00_3,SP00_4,STATUTP_1,STATUTP_2,STATUTP_3,STATUTP_4,STATUT_1,STATUT_2,STATUT_3,STATUT_4,EXTRI13,EXTRI113,EXTRI1613,changement_STC_2,changement_STC_3,changement_STC_4,changement_STATUT_2,changement_STATUT_3,changement_STATUT_4,changement_statut
id_indiv,,,,,,,,,,,,,,,,,,,,,,,,,,
G077Z53B1,1,1,1,1,1,1,1,1,NaN,NaN,NaN,NaN,11,11,11,11,1355.767390,0,0,False,False,False,False,False,False,False
G077Z84B2,NaN,1,NaN,NaN,6,1,NaN,1,NaN,NaN,NaN,NaN,NaN,11,NaN,NaN,1104.184675,0,0,True,True,True,True,True,True,True
G0874XHC1,1,1,1,1,1,1,1,1,NaN,NaN,NaN,NaN,11,11,11,11,2355.716205,0,0,False,False,False,False,False,False,False
G097435C1,2,2,NaN,2,1,1,NaN,1,NaN,NaN,NaN,NaN,11,11,NaN,11,954.584668,0,0,False,True,True,False,True,True,True
G0C7E79C1,1,1,1,1,1,1,1,1,NaN,NaN,NaN,NaN,11,11,11,11,1949.470159,0,0,False,False,False,False,False,False,False


In [44]:
df_indep.changement_statut.value_counts(normalize=True)


False    0.776974
True     0.223026
dtype: float64

In [45]:
df_indep.shape

(1659, 26)

## eliminer les NAN de la variable SP

In [46]:
df_indep['nb_SP_null'] = df_indep[["SP00_{}".format(trim) for trim in range(2,5)]].isnull().sum(1) 

/usr/local/lib/python2.7/site-packages/ipykernel/__main__.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':


In [47]:
df_indep[df_indep.nb_SP_null!=0].head()

,STC_1,STC_2,STC_3,STC_4,SP00_1,SP00_2,SP00_3,SP00_4,STATUTP_1,STATUTP_2,STATUTP_3,STATUTP_4,STATUT_1,STATUT_2,STATUT_3,STATUT_4,EXTRI13,EXTRI113,EXTRI1613,changement_STC_2,changement_STC_3,changement_STC_4,changement_STATUT_2,changement_STATUT_3,changement_STATUT_4,changement_statut,nb_SP_null
id_indiv,,,,,,,,,,,,,,,,,,,,,,,,,,,
G077Z84B2,NaN,1,NaN,NaN,6,1,NaN,1,NaN,NaN,NaN,NaN,NaN,11,NaN,NaN,1104.184675,0.000000,0.000000,True,True,True,True,True,True,True,1
G097435C1,2,2,NaN,2,1,1,NaN,1,NaN,NaN,NaN,NaN,11,11,NaN,11,954.584668,0.000000,0.000000,False,True,True,False,True,True,True,1
G117MALE1,2,1,NaN,1,1,1,NaN,1,NaN,NaN,NaN,NaN,33,11,NaN,11,717.855885,0.000000,0.000000,True,True,True,True,True,True,True,1
G2710XXZ3,2,NaN,1,1,1,NaN,1,1,35,NaN,NaN,NaN,35,NaN,11,11,798.631244,536.135092,268.067546,True,True,False,True,True,False,True,1
G2A1AZDZ2,2,NaN,2,2,1,NaN,1,1,11,NaN,NaN,NaN,11,NaN,11,11,680.708792,488.433484,244.216742,True,True,False,True,True,False,True,1


##### Avant pondération

In [48]:
df_indep[df_indep.nb_SP_null==0].changement_statut.value_counts(normalize=True)

False    0.846912
True     0.153088
dtype: float64

15,3% des indépendants dont les 4 situations sont d'emploi (variable SP) sont renseignées ont un changement de situation dans l'année

##### Après pondération

In [49]:
print df_indep[df_indep.nb_SP_null==0].EXTRI13.sum()/10**6
print (df_indep[df_indep.nb_SP_null==0].changement_statut * df_indep[df_indep.nb_SP_null==0].EXTRI13).sum()/10**6
(df_indep[df_indep.nb_SP_null==0].changement_statut * df_indep[df_indep.nb_SP_null==0].EXTRI13).sum()/ df_indep[df_indep.nb_SP_null==0].EXTRI13.sum()

2.14315594246
0.338635047624


0.15800765633294672

Sur les 2.14 millions d'indépendants en 2012, 338 000 ont changé de statut pour ne plus être ou devenir indépendant. Soit 15,8% es indépendants.

In [50]:
df_indep[df_indep.nb_SP_null==0][ (df_indep[df_indep.nb_SP_null==0].changement_statut) == True][
    ['SP00_1', 'SP00_2', 'SP00_3', 'SP00_4', 'changement_statut']].head()

,SP00_1,SP00_2,SP00_3,SP00_4,changement_statut
id_indiv,,,,,
G0E7OJPC3,3,3,1,1,True
G0F5T0VD1,3,1,1,3,True
G0H7YQPC1,3,3,1,1,True
G0J53APE2,3,1,1,1,True
G0W5HNAF2,1,1,3,1,True


In [51]:
print (df_indep[df_indep.nb_SP_null==0].changement_statut * df_indep[df_indep.nb_SP_null==0].EXTRI113).sum()/ df_indep[df_indep.nb_SP_null==0].EXTRI113.sum()
print (df_indep[df_indep.nb_SP_null==0].changement_statut * df_indep[df_indep.nb_SP_null==0].EXTRI1613).sum()/ df_indep[df_indep.nb_SP_null==0].EXTRI1613.sum()

0.174042863047
0.159923088861


In [52]:
df_indep[df_indep.nb_SP_null==0].changement_statut.value_counts(normalize=False)

False    1289
True      233
dtype: int64

In [53]:
(df_indep[df_indep.nb_SP_null==0].EXTRI13.sum()/10**6)

2.1431559424586624

In [54]:
# On recale les poids pour prendre en compte ceux qui ont une SP vide, toujours en admettant qu'il n'y a pas de biais de sélection...

In [55]:
df_indep.EXTRI13.sum()/10**6

2.2854623602652344

In [56]:
(df_indep[df_indep.nb_SP_null==0].EXTRI13 * (df_indep.EXTRI13.sum()/df_indep[df_indep.nb_SP_null==0].EXTRI13.sum())).sum()/10**6

2.2854623602652353

C'est bon au erreurs d'arrondis prét

In [57]:
df_indep["EXTRI"] = df_indep[df_indep.nb_SP_null==0].EXTRI13 * (df_indep.EXTRI13.sum()/df_indep[df_indep.nb_SP_null==0].EXTRI13.sum())

/usr/local/lib/python2.7/site-packages/ipykernel/__main__.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':


## Transitions

In [58]:
for trim in range(2,5):
    
    quite_indep = "quite_indep_{}".format(trim)
    devient_indep = "devient_indep_{}".format(trim)
    
    
    transition_indep_retraite = "transition_indep_retraite_{}".format(trim)
    transition_indep_inact = "transition_indep_inact_{}".format(trim)
    
    transition_chomage_indep = "transition_chomage_indep{}".format(trim)
    transition_retraite_indep = "transition_retraite_indep{}".format(trim)
    transition_inact_indep = "transition_inact_indep{}".format(trim)
    
    transition_sal_indep = "transition_sal_indep{}".format(trim)
    transition_indep_sal = "transition_indep_sal{}".format(trim)

    
    
    
    statut_minus_one = "STATUT_{}".format(trim-1)
    statut = "STATUT_{}".format(trim)

    df_indep[devient_indep] = ((df_indep[statut]==11) & (df_indep[statut_minus_one] !=11))
    df_indep[quite_indep] = ((df_indep[statut]!=11) & (df_indep[statut_minus_one] ==11))
    

    
    

    
    
    df_indep["transition_sal_indep_{}".format(trim)] = df_indep[devient_indep] * (df_indep["SP00_{}".format(trim-1)] == 1)
    df_indep["transition_chomage_indep_{}".format(trim)] = df_indep[devient_indep] * (df_indep["SP00_{}".format(trim-1)] == 3)
    df_indep["transition_retraite_indep_{}".format(trim)] = df_indep[devient_indep] * (df_indep["SP00_{}".format(trim-1)] == 4)
    df_indep["transition_inact_indep_{}".format(trim).format(trim)] = df_indep[devient_indep] * (df_indep["SP00_{}".format(trim-1)] >4 )
    df_indep["transition_etu_indep_{}".format(trim)] = df_indep[devient_indep] * (df_indep["SP00_{}".format(trim-1)] == 2)
    
    df_indep["transition_indep_sal_{}".format(trim)] = df_indep[quite_indep] * (df_indep["SP00_{}".format(trim)] == 1)    
    df_indep["transition_indep_chomage_{}".format(trim)] = df_indep[quite_indep] * (df_indep["SP00_{}".format(trim)] == 3)
    df_indep["transition_indep_retraite_{}".format(trim)] = df_indep[quite_indep] * (df_indep["SP00_{}".format(trim)] == 4)
    df_indep["transition_indep_inact_{}".format(trim).format(trim)] = df_indep[quite_indep] * (df_indep["SP00_{}".format(trim)] >4 )
    df_indep["transition_indep_etu_{}".format(trim)] = df_indep[quite_indep] * (df_indep["SP00_{}".format(trim)] == 2)
    
    

/usr/local/lib/python2.7/site-packages/ipykernel/__main__.py:23: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/usr/local/lib/python2.7/site-packages/ipykernel/__main__.py:24: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/usr/local/lib/python2.7/site-packages/pandas/computation/expressions.py:190: UserWarning: evaluating in Python space because the '*' operator is not supported by numexpr for the bool dtype, use '&' instead
  unsupported[op_str]))
/usr/local/lib/python2.7/site-packages/ipykernel/__main__.py:32: SettingWithCopyWarning: 

In [59]:
df_indep['to_chomage'] = df_indep[["transition_indep_chomage_{}".format(trim) for trim in range(2,5)]].sum(1)>0
df_indep['from_chomage'] = df_indep[["transition_chomage_indep_{}".format(trim) for trim in range(2,5)]].sum(1)>0

/usr/local/lib/python2.7/site-packages/ipykernel/__main__.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':
/usr/local/lib/python2.7/site-packages/ipykernel/__main__.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  from ipykernel import kernelapp as app


In [60]:
print df_indep.to_chomage.sum()
print df_indep.from_chomage.sum()

35
64


#### Pondéré

In [61]:
print (df_indep.to_chomage * df_indep.EXTRI).sum() /10**6
print (df_indep.from_chomage * df_indep.EXTRI).sum() /10**6

0.0511914766193
0.0907928474166


50 000 tombent au chomage, 90 000 sortent du chomage

In [155]:

df_indep['to_retraite'] =  df_indep[["transition_indep_retraite_{}".format(trim) for trim in range(2,5)]].sum(1)>0
df_indep['from_retraite'] = df_indep[["transition_retraite_indep_{}".format(trim) for trim in range(2,5)]].sum(1)>0

/usr/local/lib/python2.7/site-packages/ipykernel/__main__.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  from ipykernel import kernelapp as app
/usr/local/lib/python2.7/site-packages/ipykernel/__main__.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  app.launch_new_instance()


In [156]:
print df_indep.to_retraite.sum()
print df_indep.from_retraite.sum()
print (df_indep.to_retraite * df_indep.EXTRI).sum() /10**6
print (df_indep.from_retraite * df_indep.EXTRI).sum() /10**6

32
8
0.0390196762923
0.00935637158666


In [157]:

df_indep['to_inact'] = df_indep[["transition_indep_inact_{}".format(trim) for trim in range(2,5)]].sum(1)>0
df_indep['from_inact'] = df_indep[["transition_inact_indep_{}".format(trim) for trim in range(2,5)]].sum(1)>0

print df_indep.to_inact.sum()
print df_indep.from_inact.sum()
print (df_indep.to_inact * df_indep.EXTRI).sum() /10**6
print (df_indep.from_inact * df_indep.EXTRI).sum() /10**6

16
12
0.0224912250045
0.0177665468816


/usr/local/lib/python2.7/site-packages/ipykernel/__main__.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  from ipykernel import kernelapp as app
/usr/local/lib/python2.7/site-packages/ipykernel/__main__.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  app.launch_new_instance()


In [158]:

df_indep['to_etu'] = df_indep[["transition_indep_etu_{}".format(trim) for trim in range(2,5)]].sum(1)>0
df_indep['from_etu'] = df_indep[["transition_etu_indep_{}".format(trim) for trim in range(2,5)]].sum(1)>0

print df_indep.to_etu.sum()
print df_indep.from_etu.sum()
print (df_indep.to_etu * df_indep.EXTRI).sum() /10**6
print (df_indep.from_etu * df_indep.EXTRI).sum() /10**6

0
3
0.0
0.00286542610915


/usr/local/lib/python2.7/site-packages/ipykernel/__main__.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  from ipykernel import kernelapp as app
/usr/local/lib/python2.7/site-packages/ipykernel/__main__.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  app.launch_new_instance()


In [159]:

df_indep['to_sal'] = df_indep[["transition_indep_sal_{}".format(trim) for trim in range(2,5)]].sum(1)>0
df_indep['from_sal'] = df_indep[["transition_sal_indep_{}".format(trim) for trim in range(2,5)]].sum(1)>0

print df_indep.to_sal.sum()
print df_indep.from_sal.sum()
print (df_indep.to_sal * df_indep.EXTRI).sum() /10**6
print (df_indep.from_sal * df_indep.EXTRI).sum() /10**6


58
59
0.094067660327
0.102896354036


/usr/local/lib/python2.7/site-packages/ipykernel/__main__.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  from ipykernel import kernelapp as app
/usr/local/lib/python2.7/site-packages/ipykernel/__main__.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  app.launch_new_instance()


# Conclusion

Sur les 2.14 millions d'indépendants en 2012, 338 000 ont changé de statut pour ne plus être ou devenir indépendant. Soit 15,8% es indépendants.

Il y en a dans ceux là 94 000 quittent leur statut d'indépendant pour celui de salarié et 102 000 quittent leur emploi salarié pour devenir indépendant.
51000 qui tombent au chomage et 90 000 en sortent en devenant indépendant.
22 000 passent en inactivité et 18 000 en viennent.
39 000 passent à la retraite et 9 000 en sortent.
